# I45 : Use datetime Instead of time for Local Clocks

- Coordinated Universal Time (UTC) is the standard, time-zone-independent representation of time. UTC works great for computers that represent time as seconds since the UNIX epoch. But UTC isn't ideal for humans. Humans reference time relative to where they're currently lacated. People say "noon" or "8 am" instead of "UTC 15:00 minus 7 hours." If your program handles time, you'll probably find yourself converting time between UTC and local clocks to make it easier for humans to understand.

- Python provides two ways of accompliching time zone conversions. The old way, using the time built-in module, is disastrously error prone. The new way, using the datatime built-in module, works great with some help from the community-built package named pytz.

- You should be acquainted with both time and datetime to thoroughly understand why datetime is the best choice and time should be avoided.

** The time Module **

- The localtime function from the time built-in module lets you convert a UNIX timestamp (seconds since the UNIX epcoch in UTC) to a local time that matches the host computer's time zone (Pacific Daylight Time, in my case)

In [1]:
from time import localtime, strftime
now = 1407694710
local_tuple = localtime(now)
time_format = '%Y-%m-%d %H:%M:%S'
time_str = strftime(time_format, local_tuple)
print(time_str)

2014-08-11 03:18:30


In [2]:
local_tuple

time.struct_time(tm_year=2014, tm_mon=8, tm_mday=11, tm_hour=3, tm_min=18, tm_sec=30, tm_wday=0, tm_yday=223, tm_isdst=0)

In [3]:
from time import mktime, strptime

time_tuple = strptime(time_str, time_format)
utc_now = mktime(time_tuple)
print(utc_now)

1407694710.0


- How do you convertt local time in one time zone to local time in another? For example, say you are taking a flight between San Francisco and New York, and want to know what time it will be in San Francisco once you've arrived in New York.

- Directly manipulating the return values from the time, localtime, and strptime functions to do time zone conversions is a bad idea. Time zones change all the time due to local laws. It's too complicated  to manage yourself, especially if you want to handle every global city for flight departure and arrival.

- Many operating systems have configuration files that keep up with the time zone changes automatically. Python lets you use these time zones through the time module. For example, here I parse the departure time from the San Francisco time zone of Pacific Daylight Time:

In [13]:
parse_format = '%Y-%m-%d %H:%M:%S'
depart_sfo = '2014-05-01 15:45:16'
time_tuple = strptime(depart_sfo, parse_format)
time_str = strftime(time_format, time_tuple)
print(time_str)

2014-05-01 15:45:16


- After seeing that PDT works with the strptime function, you might also assume that other time zones known to my computer will also work. Unfortunately, this isn't the case. Instead, strptime raises an exception when it sees Eastern Daylight Time(the time zone for New York)

In [14]:
arrival_nyc = '2014-05-01 23:33:24 EDT'
time_tuple = strptime(arrical_nyc, time_format)

NameError: name 'arrical_nyc' is not defined

- The problem here is the platform-dependent nature of the time module. Its casual behavior is determined by how the underlying C functions work with the host operating system. This makes the functionality of the time module unreliable in Python. The time module fails to consistently work properly for multiple local times. Thus, you should avoid the time module for this purpose. If you must use time, only use it to convert between UTC and the host computer's local times. For all other types of conversions, use the datetime module.

** The datetime Module **

- The second option for representing times in Python is the datetime class from the datetime built-in module. Like the time module, datetime can be used to convert from the current time in UTC to local time.

- Here, I take the present time in UTC and convert it to my computer's local time:

In [15]:
from datetime import datetime, timezone

now = datetime(2014, 8, 10, 18, 18, 30)
now_utc = now.replace(tzinfo=timezone.utc)
now_local = now_utc.astimezone()
print(now_local)

2014-08-11 03:18:30+09:00


- The datetime module can also easily convert a local time back to UNIX timestamp in UTC.

In [16]:
time_str = '2014-08-10 11:18:30'
now = datetime.strptime(time_str, time_format)
time_tuple = now.timetuple()
utc_now = mktime(time_tuple)
print(utc_now)

1407637110.0


- Unlike the time module, the datetime module has facilities for reliably converting from one local time to another local time. However, datetime only provides the machinery for time zone operations with its tzinfo class and related methods. What's missing are the time zone definitions besides UTC.

- Luckily, the Python community has addressed this gap with the pytz module that's available for download from the Python Package Index. pytz contains a full database of every time zone definitions you might need.

- To use pytz effectively, you should always convert local times to UTC first. Perform any datetime operations you nned on the UTC values(such as offsetting). Then, convert to local times as a final step.

- For example, here I convery an NYC flight arrival time to a UTC datetime. Although some of these calls seem redundant, all of them are necessary when using pytz.

In [18]:
import pytz
arrival_nyc = '2014-05-01 23:33:24'
nyc_dt_naive = datetime.strptime(arrival_nyc, time_format)
eastern = pytz.timezone('US/Eastern')
nyc_dt = eastern.localize(nyc_dt_naive)
utc_dt = pytz.utc.normalize(nyc_dt.astimezone(pytz.utc))
print(utc_dt)

2014-05-02 03:33:24+00:00


- Once I have a UTC datetime, I can convert it to San Francisco local time.

In [20]:
pacific = pytz.timezone('US/Pacific')
sf_dt = pacific.normalize(utc_dt.astimezone(pacific))
print(sf_dt)

2014-05-01 20:33:24-07:00


- Just as easily, I can convery it to the local time in Nepal.

In [21]:
nepal = pytz.timezone('Asia/Katmandu')
nepal_dt = nepal.normalize(utc_dt.astimezone(nepal))
print(nepal_dt)

2014-05-02 09:18:24+05:45


- With datetime and pytz, these conversions are consistent across all environments regardless of what operating system the host computer is running.

## Things to Remember

- Avoid using the time module for translating between different timezones.
- Use the datetime built-in module along with the pytz module to reliably convert between times in different time zones.
- Always represent time in UTC and do conversions to local time as the final step before presentation.

# I46: Use built-in algorithms and data structures.

- When you're implementing Python programs that handle a non-travial amount of data, you'll eventually see slowdowns caused by the algorithmic complexity of your code. This usually isn't the result of Python's speed as a language. The issue, more likely, is that you aren't using the best algorithms and data structures for your problem.

- Luckily, the Python standard library has many of the algorithms and data structures you'll need yo use built in. Besides speed, using these common algorithms and data structures can make your life easier. Some of the most valuable tools you may want to use are tricky to implement correctly. Avoiding reimplementation of common functionality will save you time and headaches.

** Double-ended Queue **

- The deque class from the collections module is a double-ended queue. It provides constant time operations for inserting or removing items from its beginning or end. This makes it ideal for first-in-first-out (FIFO) queues.

In [23]:
from collections import deque

fifo = deque()
fifo.append(1)
x = fifo.popleft()

In [24]:
x

1

- The list built-in type also contains an ordered sequence of items like a queue. You can insert or remove items from the end of a list in constant time. But inserting or removing items from the head of a list takes linear time, which is much slower than the constant time of a deque.

** Ordered Dictionary **

- Standard dictionaries are unordered. That means a dict with the same keys and values can result in different orders of iteration. This behavior is a surprising byproduct of the way the dictionary's fast hash table is implemented.

In [28]:
from numpy import random

a = {}
a['foo'] = 1
a['bar'] = 2

# randomly populate 'b' to cause hash conflicts 
while True:
    z = random.randint(99, 1013)
    b = {}
    for i in range(z):
        b[i] = i
    b['foo'] = 1
    b['bar'] = 2
    for i in range(z):
        del b[i]
    if str(b) != str(a):
        break

KeyboardInterrupt: 

In [29]:
print(a)
print(b)
print('Equal?', a == b)

{'foo': 1, 'bar': 2}
{108: 108, 109: 109, 110: 110, 111: 111, 112: 112, 113: 113, 114: 114, 115: 115, 116: 116, 117: 117, 118: 118, 119: 119, 120: 120, 121: 121, 122: 122, 123: 123, 124: 124, 125: 125, 126: 126, 127: 127, 128: 128, 129: 129, 130: 130, 131: 131, 132: 132, 133: 133, 134: 134, 135: 135, 136: 136, 137: 137, 138: 138, 139: 139, 140: 140, 141: 141, 142: 142, 143: 143, 144: 144, 145: 145, 146: 146, 147: 147, 148: 148, 149: 149, 150: 150, 151: 151, 152: 152, 153: 153, 154: 154, 155: 155, 156: 156, 157: 157, 158: 158, 159: 159, 160: 160, 161: 161, 162: 162, 163: 163, 164: 164, 165: 165, 166: 166, 167: 167, 168: 168, 169: 169, 170: 170, 171: 171, 172: 172, 173: 173, 174: 174, 175: 175, 176: 176, 177: 177, 178: 178, 179: 179, 180: 180, 181: 181, 182: 182, 183: 183, 184: 184, 185: 185, 186: 186, 187: 187, 188: 188, 189: 189, 190: 190, 191: 191, 192: 192, 193: 193, 194: 194, 195: 195, 196: 196, 197: 197, 198: 198, 199: 199, 200: 200, 201: 201, 202: 202, 203: 203, 204: 204, 205: 205

- The OrderDict class from the collections module is a special type of dictionary that keeps track of the order in which its keys were inserted. Interating the keys of a OrderedDict has predictable bahavior. This can vastly simplify testing and debuggin by making all code deterministic.